"Sentence Features for Extractive Summarization:
1. Surface Features: Based on structure of documents or sentences, including **sentence position** in the document, the **number of words in the sentence**, and the number of quoted words in the sentence.
2. Content Features: Integrated three well-known sentence features based on content-bearing words i.e., centroid words, **signature terms**, and **high frequency words**.
3. Event Features: An **event** is comprised of an event term and associated event elements.
4. Relevance features: Incorporated to exploit inter-sentence relationships. It is assumed that: (1) sentences related to important sentences are important; (2) sentences related to many other sentences are important. The first sentence in a document or a paragraph is important, and other sentences in a document are compared with the leading ones."

(We have chosen to use the features in bold.)

-- *Extractive Summarization Using Supervised and Semi-Supervised Learning*

http://anthology.aclweb.org/C/C08/C08-1124.pdf

In [1]:
from collections import Counter
import pandas as pd
import spacy
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 640.0 kB/s eta 0:00:20
     --------------------------------------- 0.1/12.8 MB 656.4 kB/s eta 0:00:20
      --------------------------------------- 0.2/12.8 MB 1.4 MB/s eta 0:00:10
     - -------------------------------------- 0.5/12.8 MB 2.6 MB/s eta 0:00:05
     --- ------------------------------------ 1.1/12.8 MB 4.6 MB/s eta 0:00:03
     ----- ---------------------------------- 1.7/12.8 MB 6.1 MB/s eta 0:00:02
     ------- -------------------------------- 2.3/12.8 MB 7.1 MB/s eta 0:00:02
     --------- ------------------------------ 3.0/12.8 MB 7.9 MB/s eta 0:00:02
     ---------- ----------------------------- 3.3/12.8 MB 8.0 MB/s eta 0:00:02
     ---------- ----------------------------- 3.3/12.8 MB 8.0 MB/s eta 0:00:02
     ---------- ----------------------------- 3.4/12.8 MB 6.5 MB/s eta 0:00:02
     ----------------- ---------------------- 5.5/12.8 MB

In [2]:
# Load the processed and cleaned small portion of dataframes.
train_final = pd.read_csv("C:/Users/ankar/Desktop/summarization-models/final_processed-cleaned_train.csv")
val_final = pd.read_csv("C:/Users/ankar/Desktop/summarization-models/final_processed-cleaned_val.csv")
test_final = pd.read_csv("C:/Users/ankar/Desktop/summarization-models/final_processed-cleaned_test.csv")

In [3]:
# Hold the three datasets.
datasets = {'train': train_final, 'validation': val_final, 'test': test_final}

In [4]:
datasets['train'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28711 entries, 0 to 28710
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          28711 non-null  int64 
 1   article             28711 non-null  object
 2   highlights          28711 non-null  object
 3   id                  28711 non-null  object
 4   internet-free_art   28711 non-null  object
 5   internet-free_high  28711 non-null  object
 6   boiler-free_art     28711 non-null  object
 7   boiler-free_high    28711 non-null  object
dtypes: int64(1), object(7)
memory usage: 1.8+ MB


In [5]:
datasets['validation'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1337 entries, 0 to 1336
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          1337 non-null   int64 
 1   article             1337 non-null   object
 2   highlights          1337 non-null   object
 3   id                  1337 non-null   object
 4   internet-free_art   1337 non-null   object
 5   internet-free_high  1337 non-null   object
 6   boiler-free_art     1337 non-null   object
 7   boiler-free_high    1337 non-null   object
dtypes: int64(1), object(7)
memory usage: 83.7+ KB


In [6]:
datasets['test'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11490 entries, 0 to 11489
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          11490 non-null  int64 
 1   article             11490 non-null  object
 2   highlights          11490 non-null  object
 3   id                  11490 non-null  object
 4   internet-free_art   11490 non-null  object
 5   internet-free_high  11490 non-null  object
 6   boiler-free_art     11490 non-null  object
 7   boiler-free_high    11490 non-null  object
dtypes: int64(1), object(7)
memory usage: 718.3+ KB


In [7]:
#datasets['train'].head(2)
#datasets['validation'].head(2)
datasets['test']['boiler-free_art'].head(2)

0    The Palestinian Authority officially became th...
1    Never mind cats having nine lives. A stray poo...
Name: boiler-free_art, dtype: object

In [8]:
# For readability.
# pd.set_option('display.max_colwidth', 200)
pd.set_option('max_colwidth', None)
# pd.set_option('display.max_columns', None)

### Sentence Tokenization from cleaner, internet-free and boiler-free sentences. And explode the dfs.

In [9]:
# Speed up the following feature extraction, which invloves CPU-intensive processes.
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True) #  Use ALL of the laptop's CPU cores.

INFO: Pandarallel will run on 14 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [10]:
def sentence_tokenization(text):
    import spacy
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    sentences_as_sublists = [[token.text for token in sentence] for sentence in doc.sents]
    return sentences_as_sublists

#example
#t = """Luke Shaw has conceded his debut season at Manchester United has been a frustrating one and admits he doesn't know what he would do if he scored against boyhood club Chelsea on Saturday. Shaw joined United last summer in a £31.5million transfer from Southampton after an impressive campaign on the south coast, which resulted in him playing for England at the World Cup. However, injuries have plagued the 19-year-old's start to life at Old Trafford with the defender picking up his latest fitness setback during United's 2-1 FA Cup sixth-round exit at home to Arsenal last month. Luke Shaw admits he has endured a 'frustrating' debut season at Manchester United giving himself a 'C-' The 19-year-old's campaign has been beset by injuries since joining from Southampton last summer. """
#exa = sentence_tokenization(t)
#print(exa)

In [11]:
for name, df in datasets.items():
    # Tokenize sentences for articles and highlights
    df['art_tokenized'] = df['boiler-free_art'].parallel_apply(sentence_tokenization)
    df['high_tokenized'] = df['boiler-free_high'].parallel_apply(sentence_tokenization)
    # Explode the dataframe so each row contains one sentence.
    df = df.explode('art_tokenized')
    df = df.rename(columns={'art_tokenized': 'sentences'})
    df = df.reset_index(drop=True)
    datasets[name] = df

In [12]:
# Check columns + results. All ok.
for name, df in datasets.items(): print(f"Columns in {name} dataset:", df.columns)
datasets['train']['high_tokenized'].head(2)

Columns in train dataset: Index(['Unnamed: 0', 'article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'sentences', 'high_tokenized'],
      dtype='object')
Columns in validation dataset: Index(['Unnamed: 0', 'article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'sentences', 'high_tokenized'],
      dtype='object')
Columns in test dataset: Index(['Unnamed: 0', 'article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'sentences', 'high_tokenized'],
      dtype='object')


0    [[Lukas, Podolski, scores, his, first, goal, since, returning, to, Cologne, in, 2, -, 1, defeat, ., \n], [Jefferson, Farfan, and, Levan, Kobiashvili, scored, the, goals, that, gave, Schalke, win, ., \n], [The, defeat, leaves, Cologne, on, bottom, of, the, Bundesliga, table, after, five, games, .]]
1    [[Lukas, Podolski, scores, his, first, goal, since, returning, to, Cologne, in, 2, -, 1, defeat, ., \n], [Jefferson, Farfan, and, Levan, Kobiashvili, scored, the, goals, that, gave, Schalke, win, ., \n], [The, defeat, leaves, Cologne, on, bottom, of, the, Bundesliga, table, after, five, games, .]]
Name: high_tokenized, dtype: object

In [13]:
datasets['train']['sentences'].head(2)

0    [Lukas, Podolski, scored, his, first, goal, since, returning, to, Cologne, but, it, was, not, enough, as, they, crashed, to, a, defeat, 2, -, 1, against, Schalke, which, leaves, them, bottom, of, the, German, Bundesliga, .]
1                                                                                             [Podolski, celebrates, his, first, goal, since, returning, to, Cologne, ,, but, it, did, not, stop, his, side, sliding, to, defeat, .]
Name: sentences, dtype: object

In [14]:
# Remove newline token from the tokenized highlights (high_tokenized) column.
for name, df in datasets.items():
    if 'high_tokenized' in df.columns:
        df['high_tokenized'] = df['high_tokenized'].parallel_apply(lambda sentences: [[token for token in sentence if token != '\n'] for sentence in sentences])
    datasets[name] = df

In [15]:
datasets['train']['high_tokenized'].head(2) #OK.

0    [[Lukas, Podolski, scores, his, first, goal, since, returning, to, Cologne, in, 2, -, 1, defeat, .], [Jefferson, Farfan, and, Levan, Kobiashvili, scored, the, goals, that, gave, Schalke, win, .], [The, defeat, leaves, Cologne, on, bottom, of, the, Bundesliga, table, after, five, games, .]]
1    [[Lukas, Podolski, scores, his, first, goal, since, returning, to, Cologne, in, 2, -, 1, defeat, .], [Jefferson, Farfan, and, Levan, Kobiashvili, scored, the, goals, that, gave, Schalke, win, .], [The, defeat, leaves, Cologne, on, bottom, of, the, Bundesliga, table, after, five, games, .]]
Name: high_tokenized, dtype: object

## Sentence Embeddings

In [16]:
!pip install gensim

In [17]:
import gensim.downloader as api
import numpy as np

# Each row in the sentences column contains a list representing a single sentence, with each word as an element in this list.

model = api.load("word2vec-google-news-300")

def get_mean_sentence_embeddings(df, model):
    mean_embeddings = []
    
    for sentence in df['sentences']:
        if isinstance(sentence, list):
            word_embeddings = [model[word] for word in sentence if word in model.key_to_index]
            if word_embeddings:
                mean_embedding = np.mean(word_embeddings, axis=0)
            else:
                mean_embedding = np.zeros(model.vector_size)
            mean_embeddings.append(mean_embedding)
        else:
            mean_embeddings.append(np.zeros(model.vector_size))

    return mean_embeddings

for name, df in datasets.items():
    df['mean_sent_embeddings_art'] = get_mean_sentence_embeddings(df, model)
    datasets[name] = df

In [19]:
# Check columns + results. All ok.
for name, df in datasets.items(): print(f"Columns in {name} dataset:", df.columns)
#datasets['train']['mean_sent_embeddings_art'].head(2)
datasets['validation'].head()
#datasets['test'].head(2)

Columns in train dataset: Index(['Unnamed: 0', 'article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'sentences', 'high_tokenized', 'mean_sent_embeddings_art'],
      dtype='object')
Columns in validation dataset: Index(['Unnamed: 0', 'article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'sentences', 'high_tokenized', 'mean_sent_embeddings_art'],
      dtype='object')
Columns in test dataset: Index(['Unnamed: 0', 'article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'sentences', 'high_tokenized', 'mean_sent_embeddings_art'],
      dtype='object')


,Unnamed: 0,article,highlights,id,internet-free_art,internet-free_high,boiler-free_art,boiler-free_high,sentences,high_tokenized,mean_sent_embeddings_art
0,8477,"Since giving birth to her daughter Arabella four months ago, Rebecca Ferguson has become the victim of cruel taunts about her size 12 post-pregnancy body. But the singer has now hit back at her critics and vowed to resist the pressure to slim down. Speaking to Fabulous magazine, the 28-year-old former X Factor star confessed that recent comments aimed at her weight, and questions about when she'll 'drop the post-baby pounds', have left her more determined than ever to stand up for herself. Scroll down for video . Rebecca Ferguson gave birth to gorgeous baby Arabella just four months ago, but has faced constant questions, even from those close to her, about how and when she'll lose her 'baby weight' The singer says even though she's only a size 12 following her third pregnancy, there has been a lot of pressure on her as a celebrity mum to 'snap back into shape' within a few weeks (Left: Rebecca during her third trimester and right: Rebecca in a bikini last month . She said 'There's this culture where we celebrate people snapping back into shape a week after the birth, but I don't want to be one of those people - I just want to enjoy Arabella.' Rebecca, who has never named the father of Arabella publicly, also has two other children called Lillie, aged 10, and Karl, nine, with her former boyfriend and teenage sweetheart Karl Dures, 29. And the singer now admits that she regrets not having had this empowering new mindset after she had her other kids. Describing how she felt as a new mum in previous years, the Get Happy singer said, 'When I look back, I think I should have just enjoyed my babies and not worried about my weight or being skinny. 'I'm curvy, I've got thighs, but I'm not big. I eat healthily and I'm breastfeeding, so I know the weight will come off naturally.' Rebecca pictured with her two older children Lillie 10, (left) and Karl, nine, (right) Currently promoting her new album Lady Sings The Blues, it would be easy for Rebecca to feel the need to slim down for upcoming photo shoots and interviews. But while some celebrities are famed for getting their impossibly taut figures back just weeks after giving birth, Rebecca says she's not feeling that pressure this time around. Instead, she's standing up for new mums everywhere, by taking aim at the critics who try to publicly shame women into losing weight. The former X Factor contestant has refused to name and shame her weight critics, but revealed some have been a lot closer to home than she ever imagined (Pictured left: In concert at the St James Theatre, London, last month, and right: on This Morning last week) She said: 'Women's bodies are amazing, what our bodies can do is incredible so it's sad that we get distracted - all this stuff about being skinny, be this, be that, they're all distractions. 'They rob us of what we should be focusing on and that's sad.' Though the singer has refused to name and shame her own critics, she has revealed that they're a lot closer home than she ever imagined. 'I just want to enjoy Arabella,' said Rebecca, who previously worried about dropping post-baby pounds as quickly as possible (Rebecca pictured with her newborn) Rebecca also spoke of her toughest year yet during the interview, when her partner walked out after discovering she was pregnant with Arabella. She revealed that although it has been hard to live life as a single mother again, she is now moving forward and enjoying her life as a new mum. The star hasn't been left completely on her own though - her ex-partner Karl pops by regularly to help look after the children. And while Rebecca admits that many of their close friends and family would love to see them get back together, and she is open to the idea, she is enjoying being on her own for the time being. Rebecca flaunted her new curves on the red carpet at the 

In [20]:
# Check length. OK.
for name, df in datasets.items():
    print(f"--- {name.upper()} DATASET ---")
    for index, row in df.head().iterrows():
        embedding_length = len(row['mean_sent_embeddings_art'])
        print(f"Row {index}: Embedding Length = {embedding_length}")


--- TRAIN DATASET ---
Row 0: Embedding Length = 300
Row 1: Embedding Length = 300
Row 2: Embedding Length = 300
Row 3: Embedding Length = 300
Row 4: Embedding Length = 300
--- VALIDATION DATASET ---
Row 0: Embedding Length = 300
Row 1: Embedding Length = 300
Row 2: Embedding Length = 300
Row 3: Embedding Length = 300
Row 4: Embedding Length = 300
--- TEST DATASET ---
Row 0: Embedding Length = 300
Row 1: Embedding Length = 300
Row 2: Embedding Length = 300
Row 3: Embedding Length = 300
Row 4: Embedding Length = 300


In [21]:
# Clculate mean embeddings for tokenied highlights column (sublists within lists).

model = api.load("word2vec-google-news-300")

def calculate_avg_word2vec_list(sentence_list, model):
    
    avg_word2vec_list = []
    
    for sentence in sentence_list:
        # Filter out words that are not in the model's vocabulary.
        valid_words = [word for word in sentence if word in model.key_to_index]
        # Check if there are valid words before calculating the average.
        if valid_words:
            # Use numpy's vstack to vertically stack the word vectors.
            word_vectors = np.vstack([model[word] for word in valid_words])

            # Calculate the mean along the first axis (axis=0).
            avg_word2vec = np.mean(word_vectors, axis=0)

            avg_word2vec_list.append(avg_word2vec)
        else:
            # If no valid words, append a zero vector.
            avg_word2vec_list.append(np.zeros(model.vector_size))

    return avg_word2vec_list

for name, df in datasets.items():
    df['mean_sent_embeddings_high'] = df['high_tokenized'].apply(calculate_avg_word2vec_list, model=model)
    datasets[name] = df

In [22]:
datasets['train']['mean_sent_embeddings_high'].head(2)

0    [[0.13956158, 0.11025766, 0.035254844, 0.020601712, 0.02292809, -0.09172117, -0.052386943, -0.19335938, 0.11620624, 0.06234976, -0.01558744, -0.061077412, 0.045069475, 0.06829834, -0.05606989, 0.0028029222, 0.07958515, 0.10207602, 0.051513672, -0.036817405, 0.04653696, 0.078862116, 0.0033663237, -0.0021597056, 0.0034132737, -0.015256441, -0.027314406, 0.0029437726, 0.009549654, 0.065216064, -0.016995944, -0.08030348, -0.013310359, -0.06997446, -0.024423452, 0.029752292, -0.083364636, 0.07654747, 0.06257072, 0.08146785, 0.10569411, -0.030883789, 0.07677753, 0.04806284, 0.04081374, 0.044950046, 0.08163687, -0.016081883, 0.06109619, 0.094242975, -0.07481502, 0.048865683, -0.021474985, -0.0101095345, -0.031062199, 0.030146085, -0.06642503, -0.01903358, -0.102703385, -0.060528096, -0.01566256, 0.024376502, -0.063495345, -0.1263099, -0.0487577, -0.051095083, 0.12062425, 0.028493442, 0.04538668, 0.22952975, 0.035963792, -0.022932786, -0.0095402645, 0.04220229, -0.08510179, -0.07123272, 0

In [23]:
# Check length. OK
for name, df in datasets.items():
    print(f"--- {name.upper()} DATASET ---")
    for index, row in df.head().iterrows():
        # Iterate over each sublist in the 'mean_sent_embeddings_high' list.
        for sublist_index, sublist in enumerate(row['mean_sent_embeddings_high']):
            embedding_length = len(sublist)
            print(f"Row {index}, Sublist {sublist_index}: Embedding Length = {embedding_length}")


--- TRAIN DATASET ---
Row 0, Sublist 0: Embedding Length = 300
Row 0, Sublist 1: Embedding Length = 300
Row 0, Sublist 2: Embedding Length = 300
Row 1, Sublist 0: Embedding Length = 300
Row 1, Sublist 1: Embedding Length = 300
Row 1, Sublist 2: Embedding Length = 300
Row 2, Sublist 0: Embedding Length = 300
Row 2, Sublist 1: Embedding Length = 300
Row 2, Sublist 2: Embedding Length = 300
Row 3, Sublist 0: Embedding Length = 300
Row 3, Sublist 1: Embedding Length = 300
Row 3, Sublist 2: Embedding Length = 300
Row 4, Sublist 0: Embedding Length = 300
Row 4, Sublist 1: Embedding Length = 300
Row 4, Sublist 2: Embedding Length = 300
--- VALIDATION DATASET ---
Row 0, Sublist 0: Embedding Length = 300
Row 0, Sublist 1: Embedding Length = 300
Row 1, Sublist 0: Embedding Length = 300
Row 1, Sublist 1: Embedding Length = 300
Row 2, Sublist 0: Embedding Length = 300
Row 2, Sublist 1: Embedding Length = 300
Row 3, Sublist 0: Embedding Length = 300
Row 3, Sublist 1: Embedding Length = 300
Row 4, S

In [24]:
# Check types.
for name, df in datasets.items():
    first_non_null_entry_art = df['mean_sent_embeddings_art'].dropna().iloc[0]
    first_non_null_entry_high = df['mean_sent_embeddings_high'].dropna().iloc[0]
    print(f"Dataset: {name}")
    print(f"Type in 'mean_sent_embeddings_art': {type(first_non_null_entry_art)}")
    print(f"Type in 'mean_sent_embeddings_high': {type(first_non_null_entry_high)}\n")

Dataset: train
Type in 'mean_sent_embeddings_art': <class 'numpy.ndarray'>
Type in 'mean_sent_embeddings_high': <class 'list'>

Dataset: validation
Type in 'mean_sent_embeddings_art': <class 'numpy.ndarray'>
Type in 'mean_sent_embeddings_high': <class 'list'>

Dataset: test
Type in 'mean_sent_embeddings_art': <class 'numpy.ndarray'>
Type in 'mean_sent_embeddings_high': <class 'list'>



## Cosine Similarity (labels)

In [ ]:
#Alternative way without 'parallel_apply' for a smaller dataset (1%).

#from sklearn.metrics.pairwise import cosine_similarity

#def calculate_cosine_similarity(sentence, highlights_avg_word2vec_list):
 #   return [cosine_similarity([sentence], [highlight_sentence])[0][0] for highlight_sentence in highlights_avg_word2vec_list]
    
#for name, df in datasets.items():
 #   df['cosine_similarity_scores'] = df.apply(lambda row: calculate_cosine_similarity(row['mean_sent_embeddings_art'], row['mean_sent_embeddings_high']), axis=1)
  #  datasets[name] = df

In [25]:
# For larger dataset.
def calculate_row_cosine_similarity(row):
    from sklearn.metrics.pairwise import cosine_similarity
    def calculate_cosine_similarity(sentence, highlights_avg_word2vec_list):
        return [cosine_similarity([sentence], [highlight_sentence])[0][0] for highlight_sentence in highlights_avg_word2vec_list]

    return calculate_cosine_similarity(row['mean_sent_embeddings_art'], row['mean_sent_embeddings_high'])

for name, df in datasets.items():
    df['cosine_similarity_scores'] = df.parallel_apply(calculate_row_cosine_similarity, axis=1)
    datasets[name] = df

In [26]:
# Check columns + results. All ok.
for name, df in datasets.items(): print(f"Columns in {name} dataset:", df.columns)
datasets['train']['cosine_similarity_scores'].head()
#datasets['validation'].head()
#datasets['test'].head(2)

Columns in train dataset: Index(['Unnamed: 0', 'article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'sentences', 'high_tokenized', 'mean_sent_embeddings_art',
       'mean_sent_embeddings_high', 'cosine_similarity_scores'],
      dtype='object')
Columns in validation dataset: Index(['Unnamed: 0', 'article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'sentences', 'high_tokenized', 'mean_sent_embeddings_art',
       'mean_sent_embeddings_high', 'cosine_similarity_scores'],
      dtype='object')
Columns in test dataset: Index(['Unnamed: 0', 'article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'sentences', 'high_tokenized', 'mean_sent_embeddings_art',
       'mean_sent_embeddings_high', 'cosine_similarity_scores'],
      dtype='object')


0      [0.8736709, 0.7377715, 0.8184136]
1      [0.82316303, 0.6298492, 0.704492]
2    [0.75521606, 0.71392655, 0.7321466]
3    [0.7498019, 0.75414014, 0.73623145]
4     [0.6759272, 0.51974016, 0.8333621]
Name: cosine_similarity_scores, dtype: object

In [27]:
datasets['train'].head(1)

,Unnamed: 0,article,highlights,id,internet-free_art,internet-free_high,boiler-free_art,boiler-free_high,sentences,high_tokenized,mean_sent_embeddings_art,mean_sent_embeddings_high,cosine_similarity_scores
0,15378,"(CNN) -- Lukas Podolski scored his first goal since returning to Cologne but it was not enough as they crashed to a defeat 2-1 against Schalke which leaves them bottom of the German Bundesliga. Podolski celebrates his first goal since returning to Cologne, but it did not stop his side sliding to defeat. The hosts had already gone behind to a second-minute Jefferson Farfan strike when Podolski, who rejoined his boyhood club from Bayern Munich in the summer, levelled matters with six minutes played. But hopes Podolski's strike would inspire the home side fell flat a minute after half-time when Levan Kobiashvili scored a winner to move Schalke up to third in the table. Defeat for Cologne leaves them rock bottom with just one point from their opening five games. Schalke's early goal came courtesy of a corner to the far post where Gerald Asamoah was waiting to head the ball back across goal. Farfan was alert to the opportunity and sent his flying header into the back of the net for his second of the campaign. The mood inside the RheinEnergie Stadion improved significantly four minutes later when Podolski latched onto a through-ball before coolly stepping around Manuel Neuer to stroke home. The decisive moment in the game arrived just a minute after the interval when the dangerous Farfan attracted the home defence in the middle of the area before laying off a pass for Kobiashvili to thrash left-footed into the net. Meanwhile, in the day's other Bundesliga match, Werder Bremen had the better of the chances but were held to a 0-0 draw by Hannover.",Lukas Podolski scores his first goal since returning to Cologne in 2-1 defeat .\nJefferson Farfan and Levan Kobiashvili scored the goals that gave Schalke win .\nThe defeat leaves Cologne on bottom of the Bundesliga table after five games .,ac2e59e27564f0c77b03513cdea6616d8dd8361d,"(CNN) -- Lukas Podolski scored his first goal since returning to Cologne but it was not enough as they crashed to a defeat 2-1 against Schalke which leaves them bottom of the German Bundesliga. Podolski celebrates his first goal since returning to Cologne, but it did not stop his side sliding to defeat. The hosts had already gone behind to a second-minute Jefferson Farfan strike when Podolski, who rejoined his boyhood club from Bayern Munich in the summer, levelled matters with six minutes played. But hopes Podolski's strike would inspire the home side fell flat a minute after half-time when Levan Kobiashvili scored a winner to move Schalke up to third in the table. Defeat for Cologne leaves them rock bottom with just one point from their opening five games. Schalke's early goal came courtesy of a corner to the far post where Gerald Asamoah was waiting to head the ball back across goal. Farfan was alert to the opportunity and sent his flying header into the back of the net for his second of the campaign. The mood inside the RheinEnergie Stadion improved significantly four minutes later when Podolski latched onto a through-ball before coolly stepping around Manuel Neuer to stroke home. The decisive moment in the game arrived just a minute after the interval when the dangerous Farfan attracted the home defence in the middle of the area before laying off a pass for Kobiashvili to thrash left-footed into the net. Meanwhile, in the day's other Bundesliga match, Werder Bremen had the better of the chances but were held to a 0-0 draw by Hannover.",Lukas Podolski scores his first goal since returning to Cologne in 2-1 defeat .\nJefferson Farfan and Levan Kobiashvili scored the goals that gave Schalke win .\nThe defeat leaves Cologne on bottom of the Bundesliga table after five games .,"Lukas Podolski scored his first goal since returning to Cologne but it was not enough as they crashed to a defeat 2-1 against S

In [28]:
# Inspect cosine similarity scores and the articles/highlights to decide on the threshold.
datasets['train'][['id', 'sentences', 'high_tokenized', 'cosine_similarity_scores']].tail(40)


,id,sentences,high_tokenized,cosine_similarity_scores
1112038,c6b2303e6ee4af78227b17243dbd0a93c3087e2f,"[Embrace, :, James, Holleran, , is, comforted, by, a, mourner, ahead, of, the, funeral, of, his, 19, -, year, -, old, daughter, Madison, ., ]","[[Madison, Holleran, ,, 19, ,, took, her, life, in, Philadelphia, on, January, 17, after, leaving, a, suicide, note, and, gifts, for, her, family, .], [The, University, of, Pennsylvania, freshman, 's, death, shocked, the, school, and, her, hometown, in, New, Jersey, .], [Her, father, said, she, was, struggling, with, school, work, ,, had, shared, her, suicidal, thoughts, with, her, family, and, was, seeing, a, psychiatrist, .], [Her, former, teacher, , Edward, G., Modica, has, posted, a, petition, on, MoveOn.org, seeking, ', Madison, Holleran, Law, ', ,, requiring, universities, to, keep, statistics, on, suicides, and, attempted, suicides, by, students, .], [It, has, more, than, 3380, signatures, .]]","[0.6680949, 0.58524823, 0.64714825, 0.5218967, 0.25912905]"
1112039,c6b2303e6ee4af78227b17243dbd0a93c3087e2f,"[Heartbreaking, :, Mourners, ,, including, James, Holleran, , leave, a, memorial, mass, for, Madison, Holleran, at, Guardian, Angel, Church, in, Allendale, ,, New, Jersey, on, January, 21, ., ]","[[Madison, Holleran, ,, 19, ,, took, her, life, in, Philadelphia, on, January, 17, after, leaving, a, suicide, note, and, gifts, for, her, family, .], [The, University, of, Pennsylvania, freshman, 's, death, shocked, the, school, and, her, hometown, in, New, Jersey, .], [Her, father, said, she, was, struggling, with, school, work, ,, had, shared, her, suicidal, thoughts, with, her, family, and, was, seeing, a, psychiatrist, .], [Her, former, teacher, , Edward, G., Modica, has, posted, a, petition, on, MoveOn.org, seeking, ', Madison, Holleran, Law, ', ,, requiring, universities, to, keep, statistics, on, suicides, and, attempted, suicides, by, students, .], [It, has, more, than, 3380, signatures, .]]","[0.646528, 0.5998357, 0.44556028, 0.60113263, 0.29603457]"
1112040,c6b2303e6ee4af78227b17243dbd0a93c3087e2f,"[Days, after, Holleran, 's, death, ,, more, than, 600, devastated, relatives, and, friends, paid, their, respects, at, a, funeral, service, at, the, Guardian, Angel, Church, in, Allendale, ,, New, Jersey, .]","[[Madison, Holleran, ,, 19, ,, took, her, life, in, Philadelphia, on, January, 17, after, leaving, a, suicide, note, and, gifts, for, her, family, .], [The, University, of, Pennsylvania, freshman, 's, death, shocked, the, school, and, her, hometown, in, New, Jersey, .], [Her, father, said, she, was, struggling, with, school, work, ,, had, shared, her, suicidal, thoughts, with, her, family, and, was, seeing, a, psychiatrist, .], [Her, former, teacher, , Edward, G., Modica, has, posted, a, petition, on, MoveOn.org, seeking, ', Madison, Holleran, Law, ', ,, requiring, universities, to, keep, statistics, on, suicides, and, attempted, suicides, by, students, .], [It, has, more, than, 3380, signatures, .]]","[0.6824951, 0.6384062, 0.574244, 0.5617416, 0.39566964]"
1112041,c6b2303e6ee4af78227b17243dbd0a93c3087e2f,"[During, the, mass, ,, Mr, Holleran, said, that, his, daughter, used, to, rally, her, team, by, saying, :, ', Now, is, a, time, to, be, strong, ., ']","[[Madison, Holleran, ,, 19, ,, took, her, life, in, Philadelphia, on, January, 17, after, leaving, a, suicide, note, and, gifts, for, her, family, .], [The, University, of, Pennsylvania, freshman, 's, death, shocked, the, school, and, her, hometown, in, New, Jersey, .], [Her, father, said, she, was, struggling, with, school, work, ,, had, shared, her, suicidal, thoughts, with, her, family, and, was, seeing, a, psychiatrist, .], [Her, former, teacher, , Edward, G., Modica, has, posted, a, petition, on, MoveOn.org, seeking, ', Madison, Holleran, Law, ', ,, requiring, universities, to, keep, statistics, on, suicides, and, attempted, suicides, by, students, .], [It, has, more, than, 3380, signatures, .]]","[0.6578653, 0.531412

In [29]:
pandarallel.initialize(progress_bar=False) # Hide output becasue the next cell crashes if set to True.

INFO: Pandarallel will run on 14 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [30]:
def get_max_cosine_similarity(cosine_similarity_list):
    if cosine_similarity_list:
        return max(cosine_similarity_list)
    else:
        return 0
    
for name, df in datasets.items():
    df['max_cosine_similarity'] = df['cosine_similarity_scores'].parallel_apply(get_max_cosine_similarity)
    datasets[name] = df

In [31]:
datasets['train'][['cosine_similarity_scores', 'max_cosine_similarity']].tail(40)


,cosine_similarity_scores,max_cosine_similarity
1112038,"[0.6680949, 0.58524823, 0.64714825, 0.5218967, 0.25912905]",0.668095
1112039,"[0.646528, 0.5998357, 0.44556028, 0.60113263, 0.29603457]",0.646528
1112040,"[0.6824951, 0.6384062, 0.574244, 0.5617416, 0.39566964]",0.682495
1112041,"[0.6578653, 0.5314129, 0.725236, 0.54141057, 0.4819475]",0.725236
1112042,"[0.53882, 0.42934653, 0.53986394, 0.45693952, 0.54796517]",0.547965
1112043,"[0.6165684, 0.5082765, 0.64691746, 0.5265027, 0.40964505]",0.646917
1112044,"[0.37111458, 0.35013446, 0.4688241, 0.34386814, 0.41440636]",0.468824
1112045,"[0.5672146, 0.47128528, 0.6104403, 0.47538996, 0.41022947]",0.610440
1112046,"[0.7057005, 0.5920926, 0.50649524, 0.54206926, 0.28533745]",0.705701
1112047,"[0.4996145, 0.426006, 0.35944104, 0.5215356, 0.3578741]",0.521536


In [32]:
def mark_top_four_sentences(df):
    # Create a column for binary labels initialized to 0.
    df['binary_labels'] = 0
    # Group by article ID and process each group.
    for article_id, group in df.groupby('id'):
        # Get indices of the top 4 sentences based on max_cosine_similarity.
        top_indices = group['max_cosine_similarity'].nlargest(4).index
        df.loc[top_indices, 'binary_labels'] = 1

    return df

for name, df in datasets.items():
    datasets[name] = mark_top_four_sentences(df)

In [33]:
# Check columns + results. All ok.
for name, df in datasets.items(): print(f"Columns in {name} dataset:", df.columns)
datasets['train']['binary_labels'].tail(20)
#datasets['validation'].head()
#datasets['test'].head(2)

Columns in train dataset: Index(['Unnamed: 0', 'article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'sentences', 'high_tokenized', 'mean_sent_embeddings_art',
       'mean_sent_embeddings_high', 'cosine_similarity_scores',
       'max_cosine_similarity', 'binary_labels'],
      dtype='object')
Columns in validation dataset: Index(['Unnamed: 0', 'article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'sentences', 'high_tokenized', 'mean_sent_embeddings_art',
       'mean_sent_embeddings_high', 'cosine_similarity_scores',
       'max_cosine_similarity', 'binary_labels'],
      dtype='object')
Columns in test dataset: Index(['Unnamed: 0', 'article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'sentences', 'high_tokenized', 'mean_sent_embeddings_art',
       'mean_sent_embeddi

1112058    0
1112059    0
1112060    0
1112061    0
1112062    0
1112063    0
1112064    0
1112065    0
1112066    0
1112067    1
1112068    0
1112069    0
1112070    0
1112071    0
1112072    0
1112073    0
1112074    0
1112075    0
1112076    0
1112077    0
Name: binary_labels, dtype: int64

## Surface Features: Sentence Position
"The sentences in the earlier parts of a document are more important than
sentences in later parts."

-- p.3, *Extractive Summarization Using Supervised and Semi-Supervised Learning*

In [34]:
def sentence_position_score(row, total_sentences):
    # Calculate the position score for the sentence in its article.
    return 1 - (row['sentence_index'] / (total_sentences - 1)) if total_sentences > 1 else 1

for name, df in datasets.items():
    # Calculate the total number of sentences for each article.
    article_lengths = df.groupby('id').size()

    # Add a column for sentence index within each article.
    df['sentence_index'] = df.groupby('id').cumcount()

    # Apply the sentence position score calculation.
    df['position_score'] = df.apply(lambda row: sentence_position_score(row, article_lengths[row['id']]), axis=1)

    datasets[name] = df


In [35]:
pd.set_option("display.max_columns", None) # show all cols
pd.set_option('display.max_colwidth', None) # show full width of showing cols
pd.set_option('display.max_rows', None)

In [36]:
# Check columns + results. All ok.
for name, df in datasets.items(): print(f"Columns in {name} dataset:", df.columns)
datasets['train'][['sentence_index', 'position_score']].head(11)

Columns in train dataset: Index(['Unnamed: 0', 'article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'sentences', 'high_tokenized', 'mean_sent_embeddings_art',
       'mean_sent_embeddings_high', 'cosine_similarity_scores',
       'max_cosine_similarity', 'binary_labels', 'sentence_index',
       'position_score'],
      dtype='object')
Columns in validation dataset: Index(['Unnamed: 0', 'article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'sentences', 'high_tokenized', 'mean_sent_embeddings_art',
       'mean_sent_embeddings_high', 'cosine_similarity_scores',
       'max_cosine_similarity', 'binary_labels', 'sentence_index',
       'position_score'],
      dtype='object')
Columns in test dataset: Index(['Unnamed: 0', 'article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
     

,sentence_index,position_score
0,0,1.000000
1,1,0.888889
2,2,0.777778
3,3,0.666667
4,4,0.555556
5,5,0.444444
6,6,0.333333
7,7,0.222222
8,8,0.111111
9,9,0.000000


## Surface Features: Sentence Length

"A sentence is important if the number of words in it is within a certain range."

-- p.3, *Extractive Summarization Using Supervised and Semi-Supervised Learning*

In [37]:
# Sentence length can provide insights into the complexity and information density of a sentence, which might correlate with its summary-worthiness.
# In general, both very short and very long sentences might be less likely to be included in a summary.

def get_sentence_length(sentences):
    return len(sentences)

for name, df in datasets.items():
    # Calculate the length of each sentence
    df['sentence_length'] = df['sentences'].parallel_apply(get_sentence_length)

    # Calculate the maximum sentence length for each article
    max_lengths = df.groupby('id')['sentence_length'].transform('max')

    # Normalize the sentence lengths
    df['normalized_sentence_length'] = df['sentence_length'] / max_lengths

    datasets[name] = df

In [38]:
# Check columns + results. All ok.
for name, df in datasets.items(): print(f"Columns in {name} dataset:", df.columns)
datasets['train'][['sentence_length', 'normalized_sentence_length']].head()

Columns in train dataset: Index(['Unnamed: 0', 'article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'sentences', 'high_tokenized', 'mean_sent_embeddings_art',
       'mean_sent_embeddings_high', 'cosine_similarity_scores',
       'max_cosine_similarity', 'binary_labels', 'sentence_index',
       'position_score', 'sentence_length', 'normalized_sentence_length'],
      dtype='object')
Columns in validation dataset: Index(['Unnamed: 0', 'article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'sentences', 'high_tokenized', 'mean_sent_embeddings_art',
       'mean_sent_embeddings_high', 'cosine_similarity_scores',
       'max_cosine_similarity', 'binary_labels', 'sentence_index',
       'position_score', 'sentence_length', 'normalized_sentence_length'],
      dtype='object')
Columns in test dataset: Index(['Unnamed: 0', 'article', 'highlights', 'i

,sentence_length,normalized_sentence_length
0,35,0.813953
1,21,0.488372
2,36,0.837209
3,34,0.790698
4,17,0.395349


## Content Features: Signature terms / Named Enitites (counts)
Using the count of NEs in each sentence as a feature can provide a simple yet effective way to measure the richness and importance of the sentence. Sentences containing significant named entities (such as important people, organizations, locations, etc.) are more likely to be essential to the overall narrative or argument of the article, making them strong candidates for inclusion in summaries. For example, a sentence with a higher number of named entities (NEs) may be more informative or central for the article's main theme, and more likely to be summary-worthy.

In [ ]:
# For smaller datasets.
#nlp = spacy.load('en_core_web_sm')

#def count_named_entities_normalized(text):
 #   doc = nlp(text)
  #  ne_count = len(doc.ents)
   # total_tokens = len(doc)  # Total number of tokens in the sentence.
    #return ne_count / total_tokens if total_tokens > 0 else 0


#for name, df in datasets.items():
 #   df['normalized_NE_count'] = df['sentences'].apply(lambda x: ' '.join(x)).apply(count_named_entities_normalized)
  #  datasets[name] = df

In [ ]:
# pandarallel.initialize(progress_bar=True)

In [ ]:
# Too long for our updated dataset. Takes an hour for a 9% progress in the progress bar.

#def count_named_entities_normalized_from_tokens(tokens):
 #   import spacy
  #  nlp = spacy.load('en_core_web_sm')
    # Join tokens into a single string.
  #  text = ' '.join(tokens)
   # doc = nlp(text)
    #ne_count = len(doc.ents)
    #total_tokens = len(doc)  
    #return ne_count / total_tokens if total_tokens > 0 else 0

#for name, df in datasets.items():
 #   df['normalized_NE_count'] = df['sentences'].parallel_apply(count_named_entities_normalized_from_tokens)
  #  datasets[name] = df


In [40]:
# Check columns + results. All ok.
for name, df in datasets.items(): print(f"Columns in {name} dataset:", df.columns)
datasets['train'].head()
#datasets['validation'].head()
#datasets['test'].head()

Columns in train dataset: Index(['Unnamed: 0', 'article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'sentences', 'high_tokenized', 'mean_sent_embeddings_art',
       'mean_sent_embeddings_high', 'cosine_similarity_scores',
       'max_cosine_similarity', 'binary_labels', 'sentence_index',
       'position_score', 'sentence_length', 'normalized_sentence_length'],
      dtype='object')
Columns in validation dataset: Index(['Unnamed: 0', 'article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'sentences', 'high_tokenized', 'mean_sent_embeddings_art',
       'mean_sent_embeddings_high', 'cosine_similarity_scores',
       'max_cosine_similarity', 'binary_labels', 'sentence_index',
       'position_score', 'sentence_length', 'normalized_sentence_length'],
      dtype='object')
Columns in test dataset: Index(['Unnamed: 0', 'article', 'highlights', 'i

,Unnamed: 0,article,highlights,id,internet-free_art,internet-free_high,boiler-free_art,boiler-free_high,sentences,high_tokenized,mean_sent_embeddings_art,mean_sent_embeddings_high,cosine_similarity_scores,max_cosine_similarity,binary_labels,sentence_index,position_score,sentence_length,normalized_sentence_length
0,15378,"(CNN) -- Lukas Podolski scored his first goal since returning to Cologne but it was not enough as they crashed to a defeat 2-1 against Schalke which leaves them bottom of the German Bundesliga. Podolski celebrates his first goal since returning to Cologne, but it did not stop his side sliding to defeat. The hosts had already gone behind to a second-minute Jefferson Farfan strike when Podolski, who rejoined his boyhood club from Bayern Munich in the summer, levelled matters with six minutes played. But hopes Podolski's strike would inspire the home side fell flat a minute after half-time when Levan Kobiashvili scored a winner to move Schalke up to third in the table. Defeat for Cologne leaves them rock bottom with just one point from their opening five games. Schalke's early goal came courtesy of a corner to the far post where Gerald Asamoah was waiting to head the ball back across goal. Farfan was alert to the opportunity and sent his flying header into the back of the net for his second of the campaign. The mood inside the RheinEnergie Stadion improved significantly four minutes later when Podolski latched onto a through-ball before coolly stepping around Manuel Neuer to stroke home. The decisive moment in the game arrived just a minute after the interval when the dangerous Farfan attracted the home defence in the middle of the area before laying off a pass for Kobiashvili to thrash left-footed into the net. Meanwhile, in the day's other Bundesliga match, Werder Bremen had the better of the chances but were held to a 0-0 draw by Hannover.",Lukas Podolski scores his first goal since returning to Cologne in 2-1 defeat .\nJefferson Farfan and Levan Kobiashvili scored the goals that gave Schalke win .\nThe defeat leaves Cologne on bottom of the Bundesliga table after five games .,ac2e59e27564f0c77b03513cdea6616d8dd8361d,"(CNN) -- Lukas Podolski scored his first goal since returning to Cologne but it was not enough as they crashed to a defeat 2-1 against Schalke which leaves them bottom of the German Bundesliga. Podolski celebrates his first goal since returning to Cologne, but it did not stop his side sliding to defeat. The hosts had already gone behind to a second-minute Jefferson Farfan strike when Podolski, who rejoined his boyhood club from Bayern Munich in the summer, levelled matters with six minutes played. But hopes Podolski's strike would inspire the home side fell flat a minute after half-time when Levan Kobiashvili scored a winner to move Schalke up to third in the table. Defeat for Cologne leaves them rock bottom with just one point from their opening five games. Schalke's early goal came courtesy of a corner to the far post where Gerald Asamoah was waiting to head the ball back across goal. Farfan was alert to the opportunity and sent his flying header into the back of the net for his second of the campaign. The mood inside the RheinEnergie Stadion improved significantly four minutes later when Podolski latched onto a through-ball before coolly stepping around Manuel Neuer to stroke home. The decisive moment in the game arrived just a minute after the interval when the dangerous Farfan attracted the home defence in the middle of the area before laying off a pass for Kobiashvili to thrash left-footed into the net. Meanwhile, in the day's other Bundesliga match, Werder Bremen had the better of the chances but were held to a 0-0 draw by Hannover.",Lukas Podolski scores his first goal since returning to Cologne in 2-1 defeat .\nJefferson Farfan and Levan Kobiashvili scored the goals that gave Schalke win .\nThe defeat leaves Cologne on bottom of the Bundesliga table after five games .,"Lukas Podolski sco

## Event Features: Verbs (counts)
Verbs are integral to conveying actions and events within a text. In news articles, for instance, verbs play a crucial role in reporting what happened, when, and to whom. A higher frequency of verbs in a sentence might indicate that it is action-packed or event-rich, which could be a characteristic of summary-worthy content.

In [ ]:
# For smaller datasets.

#def count_verbs_normalized(text):
 #   doc = nlp(text)
  #  verb_count = sum(1 for token in doc if token.pos_ == 'VERB')
   # total_tokens = len(doc)  # Total number of tokens in the sentence.
    #return verb_count / total_tokens if total_tokens > 0 else 0

#for name, df in datasets.items():
    # Normalize the verb counts for each sentence
 #   df['normalized_verb_count'] = df['sentences'].apply(lambda x: ' '.join(x)).apply(count_verbs_normalized)

  #  datasets[name] = df

In [ ]:
# For larger datasets. but takes too much time, like NE counts

#def count_verbs_normalized_from_tokens(tokens):
    # Load the SpaCy model inside the function
 #   import spacy
  #  nlp = spacy.load('en_core_web_sm')

    # Join tokens into a single string
   # text = ' '.join(tokens)
#    doc = nlp(text)
 #   verb_count = sum(1 for token in doc if token.pos_ == 'VERB')
  #  total_tokens = len(doc)  # Total number of tokens in the text
   # return verb_count / total_tokens if total_tokens > 0 else 0

#for name, df in datasets.items():
 #   df['normalized_verb_count'] = df['sentences'].parallel_apply(count_verbs_normalized_from_tokens)
  #  datasets[name] = df

In [45]:
# Check columns + results. All ok.
for name, df in datasets.items(): print(f"Columns in {name} dataset:", df.columns)
datasets['train'][['sentence_length', 'normalized_sentence_length']].head(6)
#datasets['validation'].head(6)
#datasets['test'].head(2)

Columns in train dataset: Index(['Unnamed: 0', 'article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'sentences', 'high_tokenized', 'mean_sent_embeddings_art',
       'mean_sent_embeddings_high', 'cosine_similarity_scores',
       'max_cosine_similarity', 'binary_labels', 'sentence_index',
       'position_score', 'sentence_length', 'normalized_sentence_length'],
      dtype='object')
Columns in validation dataset: Index(['Unnamed: 0', 'article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'sentences', 'high_tokenized', 'mean_sent_embeddings_art',
       'mean_sent_embeddings_high', 'cosine_similarity_scores',
       'max_cosine_similarity', 'binary_labels', 'sentence_index',
       'position_score', 'sentence_length', 'normalized_sentence_length'],
      dtype='object')
Columns in test dataset: Index(['Unnamed: 0', 'article', 'highlights', 'i

,sentence_length,normalized_sentence_length
0,35,0.813953
1,21,0.488372
2,36,0.837209
3,34,0.790698
4,17,0.395349
5,26,0.604651


In [44]:
# check how many columns created so far. All ok.
for name, df in datasets.items():
    print(f"THe number of columns in the {name} dataset is", len(df.columns), "and these are: ", df.columns)

THe number of columns in the train dataset is 19 and these are:  Index(['Unnamed: 0', 'article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'sentences', 'high_tokenized', 'mean_sent_embeddings_art',
       'mean_sent_embeddings_high', 'cosine_similarity_scores',
       'max_cosine_similarity', 'binary_labels', 'sentence_index',
       'position_score', 'sentence_length', 'normalized_sentence_length'],
      dtype='object')
THe number of columns in the validation dataset is 19 and these are:  Index(['Unnamed: 0', 'article', 'highlights', 'id', 'internet-free_art',
       'internet-free_high', 'boiler-free_art', 'boiler-free_high',
       'sentences', 'high_tokenized', 'mean_sent_embeddings_art',
       'mean_sent_embeddings_high', 'cosine_similarity_scores',
       'max_cosine_similarity', 'binary_labels', 'sentence_index',
       'position_score', 'sentence_length', 'normalized_sentence_length'],
      dtype='obje

In [46]:
for name, df in datasets.items(): 
    print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1112078 entries, 0 to 1112077
Data columns (total 19 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   Unnamed: 0                  1112078 non-null  int64  
 1   article                     1112078 non-null  object 
 2   highlights                  1112078 non-null  object 
 3   id                          1112078 non-null  object 
 4   internet-free_art           1112078 non-null  object 
 5   internet-free_high          1112078 non-null  object 
 6   boiler-free_art             1112078 non-null  object 
 7   boiler-free_high            1112078 non-null  object 
 8   sentences                   1112078 non-null  object 
 9   high_tokenized              1112078 non-null  object 
 10  mean_sent_embeddings_art    1112078 non-null  object 
 11  mean_sent_embeddings_high   1112078 non-null  object 
 12  cosine_similarity_scores    1112078 non-null  object 
 1

In [48]:
columns_to_drop = ['high_sent_embeddings', 'article', 'highlights', 'internet-free_art', 
                   'internet-free_high', 'sentence_length', 'cosine_similarity_scores', 
                   'mean_sent_embeddings_high', 'boiler-free_art', 'boiler-free_high']

for name, df in datasets.items():
    df.drop(columns=[col for col in columns_to_drop if col in df.columns], axis=1, inplace=True)

In [49]:
for name, df in datasets.items(): 
    print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1112078 entries, 0 to 1112077
Data columns (total 10 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   Unnamed: 0                  1112078 non-null  int64  
 1   id                          1112078 non-null  object 
 2   sentences                   1112078 non-null  object 
 3   high_tokenized              1112078 non-null  object 
 4   mean_sent_embeddings_art    1112078 non-null  object 
 5   max_cosine_similarity       1112078 non-null  float64
 6   binary_labels               1112078 non-null  int64  
 7   sentence_index              1112078 non-null  int64  
 8   position_score              1112078 non-null  float64
 9   normalized_sentence_length  1112078 non-null  float64
dtypes: float64(3), int64(3), object(4)
memory usage: 84.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44844 entries, 0 to 44843
Data columns (total 10 columns):
 #

In [52]:
datasets['train'].head()

,Unnamed: 0,id,sentences,high_tokenized,mean_sent_embeddings_art,max_cosine_similarity,binary_labels,sentence_index,position_score,normalized_sentence_length
0,15378,ac2e59e27564f0c77b03513cdea6616d8dd8361d,"[Lukas, Podolski, scored, his, first, goal, since, returning, to, Cologne, but, it, was, not, enough, as, they, crashed, to, a, defeat, 2, -, 1, against, Schalke, which, leaves, them, bottom, of, the, German, Bundesliga, .]","[[Lukas, Podolski, scores, his, first, goal, since, returning, to, Cologne, in, 2, -, 1, defeat, .], [Jefferson, Farfan, and, Levan, Kobiashvili, scored, the, goals, that, gave, Schalke, win, .], [The, defeat, leaves, Cologne, on, bottom, of, the, Bundesliga, table, after, five, games, .]]","[0.11470295, 0.075837694, 0.035587706, 0.01364241, -0.03404762, -0.080190465, -0.009287867, -0.12817383, 0.075314224, 0.10144832, 0.014489535, -0.07114384, 0.010094873, 0.051844105, -0.08103456, 0.047095988, 0.05078651, 0.11288952, 0.040007494, -0.01411563, 0.01991364, 0.07349791, 0.015920179, -0.0034584834, 0.067733236, -0.034187052, -0.06529078, 0.013993888, 0.020629358, 0.041065086, -0.032926865, -0.027613146, -0.016833074, -0.075299494, 0.009965568, 0.04200797, -0.028869629, 0.03187403, 0.04321618, 0.054114375, 0.101246804, -0.060622644, 0.1292935, 0.004960159, 0.019059017, 0.0054305503, 0.06652148, -0.049942147, 0.083613954, 0.05633545, -0.05737252, 0.041719634, -0.005005935, -0.02816299, -0.053125843, 0.03927928, -0.047921542, -0.024672937, -0.05697474, -0.04516233, -0.04006853, 0.031866666, -0.09612406, -0.100002944, -0.071878366, -0.047021996, 0.08755046, 0.055411242, -0.0044618803, 0.17888772, 0.025567416, 0.037632383, 0.01860099, 0.0021083436, -0.1098717, -0.084859915, 0.08041592, -0.016483702, 0.04156073, 0.02110343, -0.01001608, -0.16271973, -0.009807192, -0.002482447, 0.045052364, -0.023081813, -0.013436153, 0.103066415, -0.0020857186, -0.0045860554, 0.0072289826, -0.020229997, -0.054310765, 0.0021333366, -0.043879017, -0.121284746, -0.01109998, -0.04515865, -0.024664517, 0.039232977, ...]",0.873671,1,0,1.000000,0.813953
1,15378,ac2e59e27564f0c77b03513cdea6616d8dd8361d,"[Podolski, celebrates, his, first, goal, since, returning, to, Cologne, ,, but, it, did, not, stop, his, side, sliding, to, defeat, .]","[[Lukas, Podolski, scores, his, first, goal, since, returning, to, Cologne, in, 2, -, 1, defeat, .], [Jefferson, Farfan, and, Levan, Kobiashvili, scored, the, goals, that, gave, Schalke, win, .], [The, defeat, leaves, Cologne, on, bottom, of, the, Bundesliga, table, after, five, games, .]]","[0.10943065, 0.09664457, 0.03386733, 0.026137408, -0.016226824, -0.08258595, -0.013872932, -0.18560432, 0.093229406, 0.03232709, -0.01847211, -0.10867848, 0.009917764, 0.06313548, -0.07483067, 0.03030732, 0.07075411, 0.10902674, 0.115575455, -0.052046157, 0.038380343, 0.13636331, -0.0018382353, -0.0043011834, 0.03845933, -0.03297155, -0.018965777, 0.04153263, 0.08156092, 0.027806899, -0.008565524, -0.038485806, -0.036301557, 0.009051154, 0.042114258, 0.018177705, -0.015090045, 0.002532959, 0.03969125, 0.045721617, 0.084190816, -0.05550968, 0.16026396, -0.006606158, 0.005577985, 0.039450254, 0.009426341, -0.013201769, 0.11293299, 0.05029297, -0.023214003, 0.09192433, -0.0001077091, 0.010607551, 0.012242935, 0.05610208, -0.06297302, -0.04613293, -0.023265166, -0.07540086, -0.009399414, 0.068258844, -0.07126034, -0.1408835, -0.022717644, -0.03403013, 0.0645532, 0.011725482, -0.035328586, 0.21449909, 0.058403462, -0.0014720244, 0.030047249, 0.05234662, -0.112836055, -0.115231685, 0.075868495, 0.017412972, 0.03304874, 0.04683012, -0.03270228, -0.14306742, 0.016286064, -0.035761215, 0.061609603, -0.048142377, -0.033554975, 0.0216695, 0.003209731, 0.0024557675, 0.028409172, -0.05585076, -0.046402875, 0.021154515, -0.053940717, -0.07708292, -0.018946031, -0.024581011, -0.06407973, 0.042293772, ...]",0.823163,1,1,0.888889,0.488372
2,15378,ac2e59e27564f0c77b03513cdea6616d8dd8361d,"[The, hosts, had, already, gone, behind, to

## Create the train input for ML training. 
Keep only relevant columns for machine learning binary classification.

In [53]:
columns_to_keep = ['id', 'sentence_index', 'mean_sent_embeddings_art', 
                   'max_cosine_similarity', 'position_score',  
                   'normalized_sentence_length', 'binary_labels']

# Creating new dataframes with only the specified columns.
machinelearning_datasets = {}
for name, df in datasets.items():
    machinelearning_datasets[name] = df[columns_to_keep].copy()

# Save the new dataframes to CSV files.
for name, df in machinelearning_datasets.items():
    file_name = f"final_{name}_machinelearning_features.csv"
    df.to_csv(file_name, index=False)
    
# TIME: 30'

In [54]:
machinelearning_datasets['test'].head()

,id,sentence_index,mean_sent_embeddings_art,max_cosine_similarity,position_score,normalized_sentence_length,binary_labels
0,f001ec5c4704938247d27a44948eebb37ae98d01,0,"[0.01863037, 0.007519531, 0.045720827, -0.026210938, -0.016850585, -0.12905762, 0.05534668, -0.051710814, 0.08017578, 0.08408692, 0.018985596, 0.014121094, 0.00055541995, 0.06067627, -0.14268342, 0.0023498535, 0.012729492, 0.07739258, 0.03246643, 0.012792969, 0.008016128, 0.032766115, -0.0059484863, 0.059751738, 0.03687866, -0.070896454, -0.061679993, 0.07059326, 0.011687622, 0.062841795, 0.06010498, -0.078544006, -0.048083495, 0.065682374, 0.012094727, -0.01517334, 0.0013354492, 0.0106748585, 0.007697754, 0.06078247, 0.00064575195, -0.013206787, 0.034870606, 0.0022258759, -0.023394775, -0.067088015, -0.031342775, -0.012390137, -0.039172363, 0.09900635, 0.013930664, 0.03378662, -0.00018798828, 0.010806885, -0.045269776, 0.06594696, -0.099726565, -0.050844725, -0.032305297, -0.076411135, -0.05074951, 0.046586916, 0.007199707, -0.0377417, -0.065104984, 0.0017022705, 0.044594727, 0.10273926, -0.03487915, -0.042370606, 0.002319336, 0.01404419, 0.06896485, 0.043642577, 0.005576172, -0.11226318, 0.10727539, 0.052562255, 0.12300293, 0.06959167, 0.014470215, -0.07764892, 0.028283691, 0.059782714, -0.031777345, -0.11373596, -0.059125975, 0.054311525, 0.04573303, -0.0031396484, 0.08372071, 0.003527832, -0.078933716, -0.1238977, -0.051153563, -0.019090882, 0.051157836, 0.049095612, 0.00028167723, -0.00041919708, ...]",0.827125,1.000000,0.638298,1
1,f001ec5c4704938247d27a44948eebb37ae98d01,1,"[-0.026345147, 0.072157115, 0.086598925, 0.02704027, -0.0095342, -0.07687039, 0.016142104, -0.12132263, 0.11073409, 0.10550944, 0.0004035102, -0.059637282, 0.04836358, 0.062689885, -0.08909183, 0.008592393, 0.022254096, 0.016411675, -0.05710178, -0.028415255, -0.0023735894, 0.08509488, -0.018029107, 0.026077695, 0.06347656, -0.060458712, -0.07486386, 0.06870524, 0.054563735, 0.024275038, 0.040771484, -0.09765625, -0.061143663, 0.03068712, 0.011206733, -0.0019599067, -0.059258357, 0.009733412, 0.028449165, -0.03638649, 0.040881686, -0.027676053, 0.039364286, 0.033466656, 0.038533527, -0.010462443, -0.025438096, 0.02178955, -0.024007162, 0.14838324, 0.05429416, -0.013916016, -0.0047573512, -0.05378045, -0.04107666, 0.059702132, -0.13191731, -0.06272295, -0.03536966, -0.06548394, -0.10267809, 0.020063613, -0.07535807, -0.13737996, -0.039523654, 0.03265381, 0.014987522, 0.09775119, -0.0018920898, 0.028137207, 0.03901503, 0.028355546, 0.07041422, 0.05050998, -0.0011342367, -0.051596746, 0.15212673, 0.09789361, 0.06604004, 0.15382893, 0.04693773, -0.1507348, 0.014619615, 0.026352776, 0.007529365, -0.14176051, -0.0871582, 0.006863064, 0.025482178, 0.025767008, 0.05051337, -0.006391737, -0.06198035, -0.062211778, -0.011873033, -0.0134887695, 0.0381741, -0.025986565, -0.059024386, 0.0056508384, ...]",0.613928,0.961538,0.468085,0
2,f001ec5c4704938247d27a44948eebb37ae98d01,2,"[0.023723347, 0.05842082, 0.073261514, -0.0004679362, -0.035168458, -0.10735931, 0.026655579, -0.042024232, 0.06806844, 0.10252278, -0.035551976, -0.0163208, -0.046455894, 0.036691286, -0.12680486, -0.026194254, -0.006606038, 0.029671224, 0.05464274, 0.016045125, -0.010900879, 0.012538147, -0.017256673, 0.003009669, 0.03470459, -0.020236524, -0.052922565, 0.048105877, 0.0011123657, 0.02070516, 0.058365885, -0.107573256, -0.093984984, 0.04512749, -0.002255249, -0.014235433, -0.014418539, -0.018254599, 0.004402669, 0.06399231, 0.05887451, 0.018044282, 0.06647949, -0.014736303, -0.0051574707, -0.020678965, -0.018784586, 0.0013926189, -0.024125163, 0.082457475, 0.048760988, -0.020689901, -0.045590464, -0.0068850834, -0.08009898, 0.072246805, -0.10896301, -0.05167338, -0.04591592, -0.062778726, -0.023938498, 0.09613037, -0.009847005, -0.06256307, -0.012492879, -0.0050877887, -0.041153207, 0.07437337, -0.033294678, 0.03435669, -0.001558431, -0.0043986, 0.042836506, 0.05767924, 0.0016510009, -0.13587646, 0.07966105, 0.072630815, 0.

In [56]:
feature_columns = ['id', 'sentence_index', 'mean_sent_embeddings_art', 
                   'max_cosine_similarity', 'position_score', 
                   'normalized_sentence_length']

# Create the features dataframe.
test_features_df = machinelearning_datasets['test'][feature_columns]

# Create the labels dataframe.
test_labels_df = machinelearning_datasets['test'][['binary_labels']]

In [57]:
test_features_df.to_csv('final_test_features.csv', index=False)

test_labels_df.to_csv('final_test_labels.csv', index=False)

# TIME: 8'

In [ ]:
# test_features_df.head()

In [58]:
feature_columns = ['mean_sent_embeddings_art', 
                   'max_cosine_similarity', 'position_score', 
                   'normalized_sentence_length']

# Initialize dictionaries to hold the new dataframes
train_val_features = {}
train_val_labels = {}

# Splitting the train and validation dataframes
for name in ['train', 'validation']:
    # Create the features dataframe
    train_val_features[name] = machinelearning_datasets[name][feature_columns]

    # Create the labels dataframe
    train_val_labels[name] = machinelearning_datasets[name][['binary_labels']]


In [59]:
train_val_features.keys()

dict_keys(['train', 'validation'])

In [60]:
# Saving the feature dataframes
for name, df in train_val_features.items():
    file_name = f"final_{name}_features.csv"
    df.to_csv(file_name, index=False)

# Saving the label dataframes
for name, df in train_val_labels.items():
    file_name = f"final_{name}_labels.csv"
    df.to_csv(file_name, index=False)
    
# TIME: 20'

In [61]:
# Final_test df

columns_to_keep = ['id', 'sentences', 'high_tokenized']

test_df_summaries = datasets['test'][columns_to_keep]

test_df_summaries.to_csv("summary_generation.csv", index=False)


## Future Work

In [ ]:
# thematic word: the top n words with the highest frequency in the cleaned text article.
# It shows that eight out of 10 of the top 10 words are contained in the summary while decrease harshly after the top 10 words. Therefore, the thematic
 # words were set as the top 10 words in the thematic word feature extraction
 # THE TOP 10 WORDS WITH THE HIGHEST FREQUENCY IN THE CLEANED TEXT ORDERED IN DESCDENDING ORDER. NAI
 # FIND TOP 10 verbs FOR THIS ARTICLE